### Import all necessary libraries

In [ ]:
# Importing essential libraries
import pandas as pd  # For handling CSV files
import numpy as np   # For matrix operations
import random

# PyTorch for deep learning
import torch
import torch.nn as nn  # Neural network module
import torch.nn.functional as F  # Neural network functions

# torchvision for image processing and augmentation
import torchvision.transforms as transforms

# Setting up a random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

# Suppressing minor warnings to keep the output clean
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [ ]:
class Config:
    # Updated the seed year to the current year for relevance
    seed = 2024
    
    # Updated image transformation method for more flexibility
    # Now includes both resizing and normalization steps
    image_transform = transforms.Compose([
        transforms.Resize((512, 512)),
    ])
    
    # Number of folds in a cross-validation setup remains the same
    num_folds = 5

In [ ]:
# Initialize an empty list to store the models
models = []

# Load models from each fold
for i in range(Config.num_folds):
    model_path = f'/kaggle/input/hms-baseline-resnet34d-512-512-training-5-folds/HMS_resnet_fold{i}.pth'
    with open(model_path, 'rb') as f:
        model = torch.load(f)
        models.append(model)

# Load the additional model
additional_model_path = "/kaggle/input/hms-baseline-resnet34d-512-512-training/HMS_resnet.pth"
with open(additional_model_path, 'rb') as f:
    additional_model = torch.load(f)
models.append(additional_model)

In [ ]:
def seed_everything(seed):
    # Set the seed for generating random numbers in PyTorch
    torch.manual_seed(seed)
    
    # Ensure that the PyTorch's CuDNN backend behaves deterministically
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Disabling this for reproducibility

    # Set the seed for generating random numbers in NumPy
    np.random.seed(seed)

    # Set the seed for the default Python random number generator
    random.seed(seed)

seed_everything(Config.seed)

In [ ]:
def load_and_preprocess_data(path):
    eps = 1e-6
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T  # Transpose and remove first column
    data = data[:, :300]  # Select first 300 columns
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)  # Log transformation

    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    normalized_data = (data - data_mean) / (data_std + eps)

    # Convert to PyTorch tensor and apply transformations
    data_tensor = torch.unsqueeze(torch.Tensor(normalized_data), dim=0)
    return Config.image_transform(data_tensor)

In [ ]:
def predict(models, data):
    test_preds = []
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_preds.append(pred)
    return np.array(test_preds).mean(axis=0)

In [ ]:
def prepare_submission(submission_file, test_df, test_preds):
    submission = pd.read_csv(submission_file)
    labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']
    for i, label in enumerate(labels):
        submission[f'{label}_vote'] = test_preds[:, i]
    return submission

In [ ]:
# Main execution
test_df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
submission = submission.merge(test_df, on='eeg_id', how='left')
submission['path'] = submission['spectrogram_id'].apply(
    lambda x: f"/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/{x}.parquet"
)

In [ ]:
paths = submission['path'].values
test_preds = [predict(models, load_and_preprocess_data(path)) for path in paths]

In [ ]:
final_submission = prepare_submission(
    "/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv", 
    test_df, 
    np.array(test_preds)
)

In [ ]:
final_submission.to_csv("submission.csv", index=None)
final_submission.head()

btw thanks Inspire notebook YUNSUXIAOZI

### Created and Updated by JillaniSoftTech